---
title: 10장 실전 문제 p298
date: 2022/11/01
updated: last-modified
format:
    html:
        code-fold: true
        code-tools: true
        
---

## 팀 결성 p298

서로소 집합 알고리즘 문제

1. n은 최대 노드 개수. parent 변수 크기를 정할 때 사용
2. m은 입력 받을 연산의 개수이다.
3. 입력값 중 첫 번째 숫자가 0이면 합치기 연산 수행 -> union_parent() 함수 실행
4. 입력값 중 첫 번째 숫자가 1이면 같은 팀 여부 확인. a, b가 같은 팀이면 YES, 다른 팀이면 NO 반환 -> find_parent()를 각각 실행해서 반환값이 같으면 YES, 다르면 NO 출력


In [1]:
def find_parent(parent, x):
    if parent[x] != x:
        return find_parent(parent, parent[x])
    return parent[x]

def union_parent(parent, a, b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b

n, m = map(int, input().split()) 
parent = [0] * (n + 1)
for i in range(1, n + 1):
    parent[i] = i

result = ''
for _ in range(m):
    k, a, b = map(int, input().split())
    if k == 0:
        union_parent(parent, a, b)
    elif k == 1:
        if find_parent(parent, a) == find_parent(parent, b):
            print('YES')
        else:
            print('NO')


NO
NO
YES


## 도시 분할 계획 p300

### 문제 입출력

- 입력: 집(노드) n, 길(간선) m
- 입력: [[노드, 노드, 비용], ... ]
- 출력: 비용의 최솟값 출력

### 접근 과정

1. 변수 선언
    1. n, m <- 노드, 간선 개수
    1. graph <- 간선 개수만큼 for문을 돌려 연결된 노드와 비용을 입력받는다. 최소비용 순으로 정렬하기 위해 비용을 맨 앞으로 저장
    1. graph 정렬
    1. parent <- 부모가 누구인지를 담는 테이블 생성하고 자기 자신으로 초기화
    1. min_cost <- 출력할 비용 변수
1. 큐가 빌 때까지 while문 돌리며      <- 위상정렬과 혼동
1. 비용이 최소인 노드를 큐에 넣는다.  <- 위상정렬과 혼동
1. 큐 가장 앞의 노드를 꺼내 for문을 돌리며 연결된 노드의 부모를 찾는다. <- 위상정렬과 혼동
1. 사이클이 생기지 않으면 두 노드에 union 연산을 하고 비용을 min_cost 변수에 입력한다.

### 틀린 점

- 2개의 최소 신장 트리를 만들어야 하는 점을 생각하지 못함
- 어제 공부한 크루스칼 알고리즘을 이해하고 있지 못하다는 점을 알았다. 교재 p280부터 차근차근 다시 읽었다. 
- 최소 비용의 간선부터 그래프 집합 내에 포함시키니 자연스레 최소 비용 그래프가 만들어진다. 사이클이 발생하면 신장 트리가 아닌 게 되고 더 비용이 큰 간선이 이어지기 때문에 최소 비용 조건을 만족하지 못하게 되는 것 같다.

In [32]:
from collections import deque

def find_parent(parent, x):
    if parent[x] != x:
        parent[x] = find_parent(parent, parent[x])
    return parent[x]

def union_parent(parent, a, b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b

n, m = map(int, input().split())
print(n, m)
parent = [0] * (n + 1)
for i in range(1, n + 1):
    parent[i] = i
graph = [[] for _ in range(m)]
for i in range(m):
    a, b, cost = map(int, input().split())
    print(a, b, cost)
    graph[i] = [cost, a, b]

graph.sort()

min_cost = 0

for item in graph:
    cost, a, b = item
    if find_parent(parent, a) != find_parent(parent, b):
        union_parent(parent, a, b)
        min_cost += cost
print('min_cost = ', min_cost)

7 12
1 2 3
1 3 2
3 2 1
2 5 2
3 4 4
7 3 6
5 1 5
1 6 2
6 4 1
6 5 3
4 5 3
6 7 4
min_cost =  12


교재 힌트를 보고 최소 신장 트리를 2개의 최소 신장 트리로 분리하는 코드 구현해 보자

1. 최소 신장 그래프 정보를 담을 변수를 만들어야 함 -> [비용, 노드, 노드]
1. 비용이 가장 큰 노드 제거 max()로 인덱스를 구하고 del 인덱스
1. 출력할 최소비용은 어떻게 계산??

union 연산 결과로 만들어진 그래프를 변수에 담으려면...

In [39]:
def new_union_parent(parent, a, b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b
    return a, b   # 부모, 자신

new_graph = []

for item in graph:
    cost, a, b = item
    if find_parent(parent, a) != find_parent(parent, b):
        print('1')
        p, c = new_union_parent(parent, a, b)
        print(p, c)
        new_graph.append([cost, p, c])
        min_cost += cost

new_graph

[]

### 교재 풀이

2개의 최소 신장 트리를 만드는 방법은 최소 신장 트리를 찾은 뒤에 가장 비용이 큰 간선을 제거한다.

비용이 오름차순으로 정렬되어 있기 때문에 최소 비용 누적값에서 마지막 비용을 뺀다.

In [45]:
v, e = 7, 12
edges = [
        (1, 2, 3),
        (1, 3, 2),
        (3, 2, 1),
        (2, 5, 2),
        (3, 4, 4),
        (7, 3, 6),
        (5, 1, 5),
        (1, 6, 2),
        (6, 4, 1),
        (6, 5, 3),
        (4, 5, 3),
        (6, 7, 4)]

edges.sort()

def find_parent(parent, x):
    if parent[x] != x:
        parent[x] = find_parent(parent, parent[x])
    return parent[x]

def union_parent(parent, a, b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b

parent = [0] * (v + 1)
for i in range(1, v + 1):
    parent[i] = i

result = 0
last = 0

for edge in edges:
    cost, a, b = edge
    if find_parent(parent, a) != find_parent(parent, b):
        union_parent(parent, a, b)
        result += cost
        last = cost

print(result - last)    

13


:::{.callout-warning}
교재랑 코드가 같은 것 같은데 결과가 다르다. 어디를 놓치고 있을까? 일단 넘어가자.
:::

## 커리큘럼 p303

위상 정렬 문제다. 큐를 이용해 푼다.

- 출력: n개의 강의에 대해 수강하기까지 걸리는 각각의 최소시간을 한 줄에 하나씩 출력
- 입력: 강의 수 n(강의번호는 1부터 n까지), n개의 줄에 (강의시간 선수강의번호 -1)

### 접근 과정

1. deque 라이브러리 임포트
1. 변수 - 강의수 n (수치형)
1. 변수 - (강의시간, 선수강의번호, -1) 담을 (리스트)
1. 변수 - 진입차수 (리스트) 인덱스=강의번호, 값=진입차수
1. 변수 - 강의순서와 시간을 담을 리스트
1. 변수 - 큐 객체 
1. 진입차수가 0인 강의를 큐에 append
1. while 큐가 빌 때까지 
1. 큐의 첫 번째 원소를 꺼내어 now에 저장
1. for문으로 now와 연결된 강의 돌며
1. 연결된 노드의 진입차수 - 1
1. 진입차수가 0이면 큐에 추가

In [47]:
from collections import deque

n = int(input())
lectures = [[] for _ in range(n+1)]


[[], [], [], [], [], []]

In [48]:
for i in range(1, n + 1):
    lectures[i] = list(map(int, input().split()))
lectures

[[], [10, -1], [10, 1, -1], [4, 1, -1], [4, 3, 1, -1], [3, 3, -1]]

:::{.callout-warning}
지금 실력으론 풀 수 없는 문제다. 입력 데이터의 형태가 유동적이어서 어떻게 처리해야 할지 모르겠다. 그리고 모든 강의에 대해 강의순서를 도출해야 하는 것도 ....
:::

### 교재 풀이

In [59]:
from collections import deque
import copy

v = int(input())  # 강의 개수
indegree = [0] * (v + 1)   # 모든 강의의 진입차수 0으로 초기화 = 선수강의 없음
graph = [[] for i in range(v + 1)] # 연결된 강의 정보를 담기위한 리스트
time = [0] * (v + 1)  # 각 강의의 강의 시간 0으로 초기화

# 방향 그래프의 모든 간선 정보 입력
for i in range(1, v + 1):
    data = list(map(int, input().split()))
    time[i] = data[0]  # 입력값의 첫 번째가 강의시간
    for x in data[1:-1]:  # 맨 뒤는 -1이니 두 번째부터 뒤에서 두 번째까지 슬라이싱
        indegree[i] += 1  # 진입차수 리스트의 각 강의(인덱스)의 값에 +1. for문을 돌며 선수강의 개수만큼 더하게 됨
        graph[x].append(i) # 선수 강의(x)에 각 강의(i) 연결

# 위상 정렬

result = copy.deepcopy(time) # 수강 시간 출력 리스트 변수. result가 변화할 때 time이 변하지 않게 하기 위해
q = deque()

# 진입차수 0인 강의부터 시작
for i in range(1, v + 1):
    if indegree[i] == 0:
        q.append(i)

while q:
    now = q.popleft()
    for i in graph[now]:
        # 연결 강의 수강시간 <- max( 기존 연결 강의 수강시간, 현재 강의 기존시간 + 연결 강의 수강시간) 
        result[i] = max(result[i], result[now] + time[i]) 
        indegree[i] -= 1

        if indegree[i] == 0:
            q.append(i)

# 결과 출력
for i in range(1, v + 1):
    print(i, '번 강의 수강시간 =', result[i])

1 번 강의 수강시간 = 10
2 번 강의 수강시간 = 20
3 번 강의 수강시간 = 14
4 번 강의 수강시간 = 18
5 번 강의 수강시간 = 17


:::{.callout-important}
copy.deepcopy()를 사용하면 리스트 안의 리스트도 모두 깊은복사가 된다.

:::

In [57]:
import copy
print('=== 일반 깊은 복사(2차원에서 안 됨)===')
a = [[1, 2], [3, 4]]
b = a.copy()
b[0][0] = 100
print(a, b)

print('=== copy.deepcopy(2차원에서도 됨)===')
a = [[1, 2], [3, 4]]
c = copy.deepcopy(a)
c[0][0] = 100
print(a, c)

print('=== copy.deepcopy(3차원에서도 됨)===')
d = [[[1, 2]], [[3, 4]]]
e = copy.deepcopy(d)
e[0][0][0] = 100
print(d, e)

=== 일반 깊은 복사(2차원에서 안 됨)===
[[100, 2], [3, 4]] [[100, 2], [3, 4]]
=== copy.deepcopy(2차원에서도 됨)===
[[1, 2], [3, 4]] [[100, 2], [3, 4]]
=== copy.deepcopy(3차원에서도 됨)===
[[[1, 2]], [[3, 4]]] [[[100, 2]], [[3, 4]]]
